In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.7 MB/s eta 0:00:00


In [2]:
# Essential Code to do when running colab. Not sure why Colab expect users to memorize this.
from google.colab import drive
import torch
root_dir = '/content/drive/My Drive/Colab Notebooks/AI6127/' # Set to location of your repo
drive.mount('/content/drive')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Mounted at /content/drive


In [3]:
#import pipeline from transformers module
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli').to(device)
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

# Load the BART-large MNLI model
# classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli") # Cant use as cant load onto cuda device to speed up inference

# Load the conversations from the CSV file
df1 = pd.read_csv(root_dir + "AI6127_Final_Project/Call Centre Design 1.csv")

# Define the labels for financial investments or the financial/investment/insurance industry
candidate_labels = ['Asset', 'Bond', 'Brokerage', 'Card', 'CDO', 'CLO', 'Collectibles', 'Commodities', 'Credit', 'Cryptocurrency', 'Debt', 'Derivatives', 'Digital', 'ETF', 'Estate', 'Exchange', 'Finance', 'Financial', 'Foreign', 'Fund', 'Gold', 'Green', 'Hedge', 'High-Frequency', 'Infrastructure', 'Insurance', 'Investment', 'IPO', 'Lending', 'Limited', 'Loan', 'Management', 'Master', 'MBS', 'Merchant', 'Municipal', 'Mutual', 'Online', 'Open-End', 'Options', 'Payment', 'Pension', 'Platinum', 'Portfolio', 'Precious', 'Private', 'Processing', 'Real', 'REIT', 'Retirement', 'Reverse', 'Risk', 'Savings', 'Secondary', 'Securities', 'Silver', 'Sovereign', 'Split', 'Structured', 'Tax', 'Trading', 'Treasury', 'Venture', 'Wealth', 'Wine', 'Actuarial', 'Algorithmic', 'Commodities', 'Commercial', 'Employee', 'ESOP', 'Futures', 'Municipal', 'Quantitative', 'Risk']


# Create a list to store the probability scores for each conversation
scores = []
label_str = " " 
import time as time
# Iterate over the conversations and calculate the probability score for each one
start = time.time()


for i, conversation in enumerate(df1[" Comprehensive Transcript"]): 

    premise = conversation


    # Unused method, join all labels as one complete string label, Faster method, each loop is 0.1second
    # hypothesis = " ".join(candidate_labels) 

    # Word by word, slower about 5seconds per loop on my Colab Pro+ or 25 seconds on basic colab
    this_score = 0
    for label in candidate_labels:
        hypothesis = label 
        
        # run through pytorch model pre-trained on MNLI inference
        x = tokenizer.encode(premise, hypothesis, return_tensors='pt',
                            truncation_strategy='only_first')
        logits = nli_model(x.to(device))[0]

        # we throw away "neutral" (dim 1) and take the probability of
        # "entailment" (2) as the probability of the label being true 
        entail_contradiction_logits = logits[:,[0,2]]
        probs = entail_contradiction_logits.softmax(dim=1)
        output = probs[:,1].item()
        this_score += output
    print(f"processing row:{i}. Output score:{this_score}")
    scores.append(round(this_score,4))

print("Total Time Taken: ", time.time() - start)


# Add the scores to the conversations dataframe
df1["importance_score"] = scores

# Save the updated conversations dataframe to a new CSV file
df1.to_csv(root_dir + "AI6127_Final_Project/Call Centre Design 1.csv", index=False)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2381: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(


processing row:0. Output score:74.67677760124207
processing row:1. Output score:73.9468919634819
processing row:2. Output score:74.81873828172684
processing row:3. Output score:73.99062752723694
processing row:4. Output score:73.62589228153229
processing row:5. Output score:63.767963498830795
processing row:6. Output score:59.844989866018295


KeyboardInterrupt: ignored